In [56]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix


import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict

import time
import numpy as np

import nltk as nltk

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import string
import collections
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import urllib2
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [244]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen and t not in ['the','and','or','is','some','.']] for d in docs ]





def getDocs(filename="train.dat"):
    # open docs file and read its lines
    with open(filename, "r") as fh:
        train_lines = fh.readlines()
        
    with open("English", "r") as fh:
        english_stop_words = fh.readlines()

    if(filename=="train.dat"):
        train_labels=  [l.split()[0] for l in train_lines]
        train_documents=[process_text(' '.join(l.split()[1:]),english_stop_words) for l in train_lines]  
        #train_documents=[l.split()[1:] for l in train_lines]  
        #train_documents=filterLen(train_documents,2)


    
    return train_labels, train_documents


def getTestDocs(filename="test.dat"):
    # open docs file and read its lines
    with open(filename, "r") as fh:
        test_lines = fh.readlines()
    #train_documents=[process_text(' '.join(l.split()[1:])) for l in train_lines]  

    test_documents=[process_text(' '.join(l.split()[1:]),english_stop_words) for l in test_lines]  
    
    #test_documents=[l.split() for l in test_lines]  
    #train_documents=filterLen(test_documents,2)
    
    return test_documents




def kmer(k,list):
    f = []
    for dna in list: 
        dna=dna.lower()
        for x in range(len(dna)+1-k):
           
            kmer = dna[x:x+k]
            f.append(kmer)
    return f


def process_text(text,english_stop_words, stem=True, length=2, c=5):
    """ Tokenize text and stem words removing punctuation """
    
    text = BeautifulSoup(text,"lxml").text
    
    tokens = text.split()
    #s.translate(None, string.punctuation)
    text = text.translate(string.punctuation)
    
       
    filtered_words = [word for word in tokens if word not in english_stop_words and len(word)>2]
    filtered_words = kmer(c, filtered_words)


    return filtered_words




In [245]:
train_labels, train_documents = getDocs()



print train_documents[1:10]

[[u'movie', u'slowe', u'lower', u'molas', u'olass', u'lasse', u'asses', u'janua', u'anuar', u'nuary', u'uary.', u'ary..', u'ry...', u'alask', u'laska', u'aska.', u'togea', u'ogeat', u'geath', u'eathe', u'ather', u'previ', u'revie', u'eview', u'shoul', u'hould', u'award', u'manag', u'anagi', u'nagin', u'aging', u'every', u'secon', u'econd', u'conds', u'inter', u'nteri', u'teris', u'erist', u'risti', u'istin', u'sting', u'previ', u'revie', u'eview', u'view.', u'peopl', u'eople', u'watch', u'atchi', u'tchin', u'ching', u'with,', u'sever', u'evera', u'veral', u'times', u'imes.', u'after', u'over,', u'havin', u'aving', u'woken', u'taken', u'hopin', u'oping', u'somet', u'ometh', u'methi', u'ethin', u'thing', u'actua', u'ctual', u'tuall', u'ually', u'happe', u'appen', u'ppen,', u'nothi', u'othin', u'thing', u'does.', u'loose', u'track', u'peopl', u'eople', u"ople'", u"ple's", u'motiv', u'otive', u'tives', u'ives,', u'chara', u'harac', u'aract', u'racte', u'acter', u'cters', u'unint', u'ninte'

In [253]:
test_documents = getTestDocs("test.dat")

In [254]:
print len(test_documents)

25000


In [255]:


def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        #print cnt
        keys = list(k for k,_ in cnt.most_common())
        #print keys
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat,idx


def build_test_matrix(docs,idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    
    
    nnz = 0
    test_docs = []
    for d in docs:
        copy_d=[]
     
        for w in d:
            if w in idx:
                copy_d.append(w)
        
        nnz += len(set(copy_d))
        test_docs.append(copy_d)
        
        
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in test_docs:
        cnt = Counter(d)
        #print cnt
        keys = list(k for k,_ in cnt.most_common())
        #print keys
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)))
        
        
def splitData(mat, cls, fold=1, d=10):
    r""" Split the matrix and class info into train and test data using d-fold hold-out
    """
    n = mat.shape[0]
    r = int(np.ceil(n*1.0/d))
    mattr = []
    clstr = []
    # split mat and cls into d folds
    for f in range(d):
        if f+1 != fold:
            mattr.append( mat[f*r: min((f+1)*r, n)] )
            clstr.extend( cls[f*r: min((f+1)*r, n)] )
    # join all fold matrices that are not the test matrix
    train = sp.vstack(mattr, format='csr')
    # extract the test matrix and class values associated with the test rows
    test = mat[(fold-1)*r: min(fold*r, n), :]
    clste = cls[(fold-1)*r: min(fold*r, n)]

    return train, clstr, test, clste

def classifyNames(names, cls, c=3, k=3, d=10):
    r""" Classify names using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """
    docs = [cmer(n, c) for n in names]
    mat = build_matrix(docs)
    # since we're using cosine similarity, normalize the vectors
    csr_l2normalize(mat)
   



In [247]:
mat,word_dict = build_matrix(train_documents)
csr_l2normalize(mat)


In [256]:
tmat = build_test_matrix(test_documents,word_dict)
csr_l2normalize(tmat)
print tmat.shape


(25000, 276290)


In [248]:
print mat.shape



(25000, 276290)


In [257]:
def getNeighborsWhole(trainingSet, testSet, k):
    distances = []
    num_rows, num_cols = testSet.shape
    labels=[]
    distances=testSet.dot(trainingSet.T).todense()
    #distances=testSet.dot(trainingSet.T)
    distances =np.array(distances)
    indixes= np.argsort(-distances,axis=1)
    indixes=np.array(indixes)

 
    for row in range(num_rows):
        poscount=0
        negcount=0
        l=indixes[row]
        d=distances[row]

        for x in range(k):
            dist=d[x]

            if(dist>0):
                if(train_labels[l[x]]=='-1'):
                    negcount+=1  
                else:
                    poscount+=1
        label='+1'
        if(negcount>poscount):
            label='-1'
        elif(negcount==poscount):
            label=train_labels[l[0]]
            
        labels.append(label)
    return labels    

In [250]:
 

    
def classify(x, train, clstr,k=3):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        

        
macc = 0.0
d=100



#svd = TruncatedSVD(n_components=5000, n_iter=7, random_state=42)

#scaled_mat = svd.fit_transform(mat)




for f in range(5,200):
    
    
        # split data into training and testing
    train, clstr, test, clste = splitData(mat, train_labels, f+1, d)
        # predict the class of each test sample
    clspr = getNeighborsWhole(train,test,f)
    #clspr = [ classify(test[i,:], train, clstr,19) for i in range(test.shape[0]) ]
        # compute the accuracy of the prediction
    acc = 0.0
    for i in range(len(clste)):
        if clste[i] == clspr[i]:
            acc += 1
    acc /= len(clste)
    print str(f)+"--->"+str(acc)
    macc += acc
        
print macc/195

5--->0.532
6--->0.528
7--->0.516
8--->0.548
9--->0.544
10--->0.52
11--->0.484
12--->0.54
13--->0.52
14--->0.52
15--->0.524
16--->0.58
17--->0.612
18--->0.544
19--->0.572
20--->0.572
21--->0.6
22--->0.604
23--->0.624
24--->0.572
25--->0.616
26--->0.624
27--->0.644
28--->0.652
29--->0.572
30--->0.612
31--->0.648
32--->0.672
33--->0.648
34--->0.68
35--->0.66
36--->0.704
37--->0.656
38--->0.712
39--->0.684
40--->0.68
41--->0.708
42--->0.652
43--->0.688
44--->0.708
45--->0.716
46--->0.7
47--->0.704
48--->0.664
49--->0.704
50--->0.732
51--->0.74
52--->0.748
53--->0.752
54--->0.736
55--->0.748
56--->0.736
57--->0.736
58--->0.752
59--->0.752
60--->0.748
61--->0.724
62--->0.76
63--->0.716
64--->0.724
65--->0.776
66--->0.756
67--->0.688
68--->0.78
69--->0.756
70--->0.716
71--->0.736
72--->0.744
73--->0.768
74--->0.732
75--->0.776
76--->0.708
77--->0.752
78--->0.752
79--->0.764
80--->0.752
81--->0.768
82--->0.732
83--->0.78
84--->0.788
85--->0.764
86--->0.712
87--->0.792
88--->0.76
89--->0.748
90

ZeroDivisionError: float division by zero

In [258]:

    

import os
filename = 'output.dat'
highscore = open(filename,'w')
num_rows, num_cols = tmat.shape
print num_rows

clspr = getNeighborsWhole(mat,tmat,99)

for e in clspr:
    
    highscore.write(e+"\n")

highscore.close()  

25000


In [128]:

from itertools import product
from string import ascii_lowercase
keywords = map(''.join, product("hello", repeat=3))
print keywords

['hhh', 'hhe', 'hhl', 'hhl', 'hho', 'heh', 'hee', 'hel', 'hel', 'heo', 'hlh', 'hle', 'hll', 'hll', 'hlo', 'hlh', 'hle', 'hll', 'hll', 'hlo', 'hoh', 'hoe', 'hol', 'hol', 'hoo', 'ehh', 'ehe', 'ehl', 'ehl', 'eho', 'eeh', 'eee', 'eel', 'eel', 'eeo', 'elh', 'ele', 'ell', 'ell', 'elo', 'elh', 'ele', 'ell', 'ell', 'elo', 'eoh', 'eoe', 'eol', 'eol', 'eoo', 'lhh', 'lhe', 'lhl', 'lhl', 'lho', 'leh', 'lee', 'lel', 'lel', 'leo', 'llh', 'lle', 'lll', 'lll', 'llo', 'llh', 'lle', 'lll', 'lll', 'llo', 'loh', 'loe', 'lol', 'lol', 'loo', 'lhh', 'lhe', 'lhl', 'lhl', 'lho', 'leh', 'lee', 'lel', 'lel', 'leo', 'llh', 'lle', 'lll', 'lll', 'llo', 'llh', 'lle', 'lll', 'lll', 'llo', 'loh', 'loe', 'lol', 'lol', 'loo', 'ohh', 'ohe', 'ohl', 'ohl', 'oho', 'oeh', 'oee', 'oel', 'oel', 'oeo', 'olh', 'ole', 'oll', 'oll', 'olo', 'olh', 'ole', 'oll', 'oll', 'olo', 'ooh', 'ooe', 'ool', 'ool', 'ooo']


In [ ]:
mat1 =[[1,2][3,4]]
mat2 =[[8,9],[5,7],[1,2]]
    
    
    
    
distances=mat1.dot(mat2.T)
distances =np.array(distances)
indixes= np.argsort(distances,axis=1)[::-1]

print distances

print indixes

In [216]:
mat1 =np.array([[1,90],[3,4]])
mat2 =np.array([[8,9],[5,7],[1,2],[6,7],[8,4]])
train_labels=['-1','+1','-1','+1','-1']

clspr = getNeighborsWhole(mat2,mat1,3)

print clspr



[[818 635 181 636 368]
 [ 60  43  11  46  40]]
[[0 3 1 4 2]
 [0 3 1 4 2]]
(2, 5)
[0 3 1 4 2]
[818 635 181 636 368]
818
0
1
0
635
3
1
1
181
1
1
2
[0 3 1 4 2]
[60 43 11 46 40]
60
0
1
0
43
3
1
1
11
1
1
2
['+1', '+1']
